In [ ]:
# This takes photometry of stars through apertures arranged on a grid which moves
# to compensate for poor telescope guiding

# Created 2021 Aug. 20 by E.S.

In [1]:
import pandas as pd
import numpy as np
#from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os
from numpy.linalg import inv
import svdt

%matplotlib qt

In [2]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/00_dark_subtracted_flat_fielded/"
stem_star_positions = "/Users/bandari/Documents/git.repos/kriz_projects/notebooks_for_development/output_other/"

In [3]:
# names of files

file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

file_names_pos = sorted(glob.glob(stem_star_positions + "V1432_Aql-*dat"))

# file of measured displacements between images
file_name_disp = "displacements_v1432_aql.csv"

In [4]:
def coords_brightest(star_positions_scrambled_ids_pass, num_rank):
    '''
    Return the coordinates of the brightest stars in the dataframe (column "Flux")
    
    INPUTS:
    star_positions_scrambled_ids_pass: DataFrame with xcentroid, ycentroid, and flux info
    num_rank: Nth brightest stars returned (i.e., 3 -> coordinates of 3 brightest are returned)
    
    OUTPUTS:
    2D array of x,y coordinates of the Nth brightest stars
    '''
    
    brightest_fluxes = np.sort(star_positions_scrambled_ids_pass["flux"])[-int(num_rank):]
    #print("brightest")
    #print(brightest_fluxes)
    # pick table rows where stars are at least as bright as the third-brightest star
    idx_cond = star_positions_scrambled_ids_pass["flux"]>=brightest_fluxes[0]
    # get their positions
    x_brights = star_positions_scrambled_ids_pass["xcentroid"][idx_cond]
    y_brights = star_positions_scrambled_ids_pass["ycentroid"][idx_cond]
    #print(star_positions_scrambled_ids_pass[idx_cond])
    coords_empir_brights = zip(x_brights,y_brights)
    
    return list(coords_empir_brights)

In [5]:
def match_stars(predicted_coords_pass, empirical_coords_pass):
    '''
    Matches up predicted and empirical coordinates of bright stars
    
    INPUTS:
    predicted_coords_pass: list of predicted positions of the brightest stars in the initial frame
    empirical_coords_pass: list of empirical coordinates of LOTS of stars in the current frame
    
    OUTPUTS:
    df_map: DataFrame with mapping of predicted and true coordinates
    '''
    
    # convert inputs to dataframes
    df_pred = pd.DataFrame(predicted_coords_pass, columns=["x_pred", "y_pred"])
    df_empir = pd.DataFrame(empirical_coords_pass, columns=["x_empir", "y_empir"])
    
    # initialize columns of 'true' coordinates to predicted dataframe
    df_pred["x_true"] = np.nan
    df_pred["y_true"] = np.nan

    
    # pluck out the 'true' coordinates from the empirical data
    for num_bright in range(0,len(df_pred)):
        # condition is "where difference in coordinates is less than N=3 pixels"
        idx_cond = np.sqrt(
                        np.add(
                            np.power(np.subtract(df_pred["x_pred"][num_bright],df_empir["x_empir"]),2.),
                            np.power(np.subtract(df_pred["y_pred"][num_bright],df_empir["y_empir"]),2.)
                            )
                        ) < 3
        
        # populate the 'true' columns of the DataFrame of predicted values
        df_pred["x_true"][num_bright] = df_empir["x_empir"].loc[idx_cond].values[0]
        df_pred["y_true"][num_bright] = df_empir["y_empir"].loc[idx_cond].values[0]
        
    df_map = df_pred.copy(deep=True)
        
    return df_map

In [6]:
# read in image displacements for ALL stars in ALL the frames
disp_images = pd.read_csv(stem_star_positions+file_name_disp)
# read in first image's star locations (but with scrambled IDs)
initial_star_pos = pd.read_csv(file_names_pos[0], delim_whitespace=True)
#print(initial_star_pos)

# find the coordinates of the brightest three stars in the initial image;
# we will use this to figure out the net translation and rotation of successive frames
initial_coords_empir_brights = coords_brightest(initial_star_pos,num_rank=3) 
#print(initial_coords_empir_brights)

In [9]:
def transform_coords(T_trans, R_rot, array_pre_transform):
    '''
    Applies translation and rotation to input array.
    
    INPUTS:
    array_pre_transform: 2D array before transformation [numpy ndarray (Nx3)]
    T_trans: translation array
    R_rot: rotation array
    
    OUTPUT:
    array_post_transform: the input 2D array, after translation and rotation
    '''
    
    # rotate first
    array_post_rotate = np.dot(R,array_pre_transform.T).T # note the transposes here
    # then translate
    array_post_translate = np.stack([np.add(coord_set,T_trans) for coord_set in array_post_rotate])
    
    print("post rotate")
    print(array_post_rotate.round(decimals=2))
    print("post translate")
    print(array_post_translate.round(decimals=2))
    
    return array_post_translate

In [11]:
## ## 
# fake data
test_predicted_xyz = np.array(([-1,0,0],[0,1,0],[1,0,0],[1,0,0]))
test_true_xyz = np.array(([0,-1,0],[-1,0,0],[0,1,0]))
#test_true_xyz = np.array(([0,-1,0],[-1,0,0],[0,1,0],[0,1,0])) # rotation, no translation
#test_true_xyz = np.array(([-2,1,0],[-1,2,0],[0,1,0],[0,1,0])) # translation, no rotation
test_true_xyz = np.array(([0,-2,0],[-1,-1,0],[0,0,0],[0,0,0])) # rotation and translation
print("predicted:")
print(test_predicted_xyz)
print("true:")
print(test_true_xyz)

# generate the transformations from the data
R, T, RMSE = svdt.svd(A=test_predicted_xyz, B=test_true_xyz)

# B = R*A + L + err.

#print("R")
#print(R.round(decimals=2))
#print("T")
#print(T.round(decimals=2))

# test the function
transformed_grid_xyz = transform_coords(T_trans=T, R_rot=R, array_pre_transform=test_predicted_xyz)


print("transformed")
print(transformed_grid_xyz.round(decimals=2))

predicted:
[[-1  0  0]
 [ 0  1  0]
 [ 1  0  0]
 [ 1  0  0]]
true:
[[ 0 -2  0]
 [-1 -1  0]
 [ 0  0  0]
 [ 0  0  0]]
post rotate
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [-0.  1.  0.]
 [-0.  1.  0.]]
post translate
[[ 0. -2.  0.]
 [-1. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
transformed
[[ 0. -2.  0.]
 [-1. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [12]:
#for num_star_file in range(0,len(file_names_stars)):
for num_star_file in range(2200,2201):
    # for each frame, calculate needed shifts of the aperture grid and take photometry

    # make copy of DataFrame of initial stars (we will change the x,y coords to make a prediction)
    predicted_star_pos = initial_star_pos.copy(deep=True)

    # read in current star image
    star_image = fits.open(file_names_stars[num_star_file])[0].data

    # read in empirically-found positions and photometry of ALL stars in this frame (but with scrambled IDs)
    star_info_scrambled_ids = pd.read_csv(file_names_pos[num_star_file], delim_whitespace=True)

    # retrieve (x,y) linear displacement vectors found independently for ALL stars in that frame, as was 
    # found by cross-correlating images
    idx = disp_images["file_name"]==str(os.path.basename(file_names_stars[num_star_file]))
    x_disp = disp_images["x_off_pix"].where(idx).dropna().values[0]
    y_disp = disp_images["y_off_pix"].where(idx).dropna().values[0]

    # shift aperture grid of first (i.e., [0]) array of images by the linear displacement; this is a first-order 
    # correction to fit the current frame (n.b. we use this grid and translate it in order to preserve the ID 
    # information of the stars; otherwise, if we re-find the stars in each frame we don't know which is which)
    predicted_star_pos["xcentroid"] = initial_star_pos["xcentroid"]-x_disp
    predicted_star_pos["ycentroid"] = initial_star_pos["ycentroid"]-y_disp
    predicted_coords = list(zip(predicted_star_pos["xcentroid"],predicted_star_pos["ycentroid"]))

    # find the predicted coordinates (in this image) of the brightest three stars (as measured in the initial image)
    predicted_coords_brights = coords_brightest(predicted_star_pos,num_rank=3)

    # find the empirical coordinates of the brightest *twenty* stars in this image
    # (here we choose twenty, and from those we'll match the correct three, because the brightest
    # three stars in the first image are not necessarily the brightest three in every image)
    empirical_coords_brights = coords_brightest(star_info_scrambled_ids,num_rank=30)

    # match the stars by finding the sets of predicted and empirical coordinates that are within 3 (or so) pixels
    coord_mapping_brightest = match_stars(predicted_coords_brights, empirical_coords_brights)
    
    # use SVD decomposition to find the translation and rotation to minimize the residuals
    # Note syntax here:
        # Matrix of estimated, translated positions    A: [ [x_1 y_1 z_1=0], [x_2 y_2 z_2=0], [x_3 y_3 z_3=0] ]
        # Matrix of empirically found positions        B: [ [x_1' y_1' z_1'=0], [x_2' y_2' z_2'=0], [x_3' y_3' z_3'=0] ]
        # Rotation matrix                              R
        # Translation matrix                           T
    predicted_array = np.array([[coord_mapping_brightest["x_pred"][0], coord_mapping_brightest["y_pred"][0], 0.], 
                           [coord_mapping_brightest["x_pred"][1], coord_mapping_brightest["y_pred"][1], 0.], 
                           [coord_mapping_brightest["x_pred"][2], coord_mapping_brightest["y_pred"][2], 0.]])
    true_array = np.array([[coord_mapping_brightest["x_true"][0], coord_mapping_brightest["y_true"][0], 0.], 
                           [coord_mapping_brightest["x_true"][1], coord_mapping_brightest["y_true"][1], 0.], 
                           [coord_mapping_brightest["x_true"][2], coord_mapping_brightest["y_true"][2], 0.]])
    R, T, RMSE = svdt.svd(A=predicted_array, B=true_array)
    
    # transform the full aperture grid with predicted positions
    # translation, then rotation
    predicted_grid_xy = np.array(predicted_star_pos[["xcentroid" ,"ycentroid"]])
    # append z=0
    predicted_grid_xyz = np.append(predicted_grid_xy,np.zeros((len(predicted_star_pos["xcentroid"]),1)),1)
    
    print(type(predicted_grid_xyz))
    print(predicted_grid_xyz.shape)
    print(predicted_grid_xyz)
    transformed_grid_xyz = transform_coords(T,R,predicted_grid_xyz)
    
 
    
    #print("array post rotate")
    #print(array_post_rotate)
    '''
    print("predicted_grid_xyz")
    print(predicted_grid_xyz)
    print("T")
    print(T)
    print("array_post_translate")
    print(array_post_translate)
    print("array_post_rotate")
    print(array_post_rotate.T)
    '''
    

    # for photometry, we just want the x,y coordinates (not z)
    #print(predicted_array)
    predicted_apertures = [[coords[0],coords[1]] for coords in predicted_grid_xyz]
    #post_translation_apertures = [[coords[0],coords[1]] for coords in array_post_translate]
    #post_translation_rotate = [[coords[0],coords[1]] for coords in array_post_rotate]
    post_transform = [[coords[0],coords[1]] for coords in transformed_grid_xyz]
    #print(predicted_apertures)
    #print(post_translation_apertures)
    #print(post_translation_rotate)

    
    ## FYI

    apertures_pred = CircularAperture(predicted_apertures, r=4.)
    apertures_post_transf = CircularAperture(post_transform, r=4.)
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys_r', origin='lower', norm=norm,
               interpolation='nearest')
    apertures_pred.plot(color='red', lw=1.5, alpha=0.5)
    apertures_post_transf.plot(color='blue', lw=1.5, alpha=0.5)
    plt.show()


    '''
    PHOTOMETRY
    phot_table = aperture_photometry(star_image, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output


    ## ## START TEST
    # convert to DataFrame
    df_phot = phot_table.to_pandas()


    # write text file with aperture photometry

    text_file_name = "output_other/aper_photom_"+str(os.path.basename(file_names_stars[num_star_file])).split(".")[-2]+".dat"
    ascii.write(phot_table, text_file_name, overwrite=False)
    print("Wrote out "+text_file_name)

    # write FYI plot
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    '''


    ## ## END FOR-LOOP

<class 'numpy.ndarray'>
(302, 3)
[[ 2.65534650e+02  1.56149222e+01  0.00000000e+00]
 [-8.45250549e+01  1.89056526e+01  0.00000000e+00]
 [ 2.74566330e+02  1.77664479e+01  0.00000000e+00]
 [ 3.97172260e+02  1.88949629e+01  0.00000000e+00]
 [-7.63874965e+01  2.38088388e+01  0.00000000e+00]
 [-6.79796940e+01  2.39684733e+01  0.00000000e+00]
 [ 7.75203700e+01  2.33791306e+01  0.00000000e+00]
 [ 3.03137300e+02  2.43727916e+01  0.00000000e+00]
 [-7.21118500e+00  2.54734770e+01  0.00000000e+00]
 [-5.97740890e+01  2.70837900e+01  0.00000000e+00]
 [ 3.76691050e+02  2.75297400e+01  0.00000000e+00]
 [-7.77209860e+01  2.87043420e+01  0.00000000e+00]
 [ 3.02286960e+02  3.24907890e+01  0.00000000e+00]
 [-5.43440550e+01  3.93314890e+01  0.00000000e+00]
 [ 1.81342050e+02  4.26154490e+01  0.00000000e+00]
 [ 3.22564500e+02  4.57582500e+01  0.00000000e+00]
 [ 3.93587500e+02  4.51167560e+01  0.00000000e+00]
 [ 4.22238610e+02  4.57978270e+01  0.00000000e+00]
 [ 9.48971200e+01  4.86567010e+01  0.00000000e+00